OptionGun03_Tonka
Notebook created 12/07/22
12/14/2 - Tested and running as OptionGun03_Tonka_Prdtn
Dev version in this notebook temporarilly until I learn how to do releases.

User Notes
1. This program needs a user key from Darqube.com, which it uses to get real-time stock prices.
   Go to the Darqube website get a user account and choose the free service. Then get your key.
   Find the Dar_key line just below the "User Parameters" block in the cell below.
   Enter it between the quotes on the line:  DAR_key = ''
   
2. OptionGun produces three csv files:
     root_data.csv. This contains a list of all tickers, with real-time price at the time of execution, 
     and the netx Earnings Report date for each ticker, if available. DO NOT CHANGE THIS FILE.
     all_options_suffix.csv. This file contains every option available for every ticker in the list. 
     best_options_suffix.csv. This file contains only the options with meet the selection criteria in the program.
     
2. The parameters just under the "User Parameters" header block can be changed.
    TICKERS. YOu can add and remove ticker symbols from this list. 
         OptionGun gets option strings and real-time prices for every ticker in this list. 
    HIDE_TICKERS. You may want to collect option info for a ticker, but not see that ticker in best_file.csv.
    Just put the ticker to be hidden in this list. 
    FILE_ID_TAG is the suffix that OptionGun will attach to all_options and best_options. You can change it at any time.
    
3. OptionGun needs about 2-3 seconds to get the option string for each ticker. For 100 tickers, it will take about 5 minutes.

OptionGun Structure

- HEADERS and CONSTANTS

- Broker_Converter_Van - TEMPORARY here to get Vanguard report and produce trade_log_open for BTC offers.

- GET OPTION DATA
       - Get real-time price data from Darqube.
       - Get option strings from yfinance.
       - Get ER dates from yfinance.
       - Parse option strings into common POWMax format.

12/15/22 - Added little option guns. 
- Put_STO_Gun - Screen and select PSTO offers
- Call_STO_Gun- Screen and select CSTO offers
- Put_BTC_Gun - Screen and select PBTC offers
- Call_BTC_Gun- Screen and select CBTC offers

TODO 12/15/22 - Dev Notes
1. Convert all_options, best_options, and real-time prices to fcts. Set up calls so they can be bypassed for testing.
1. Complete end-to-end flow from each little option gun to output a csv file and into XLOptionBoard.
1. Use new column lables to split broker_report into three.
2. Delete fill-col headers. Replace with correct column headers.


In [1]:
# ****************
#
#Header and Constants
#
# *****************
# Purpose - OptionGun downlaods option strings and ER dates from yahoo yfinance.
#                It downloads real-time stock prices and company fundamental financial data from Darqube.
#                It calculates metrics for options 
#                and selects options that score the best on these metrics: POW, ARR, PctOTM, and Expiry DaysOut.
#
# Started 11/11/19 from TQIRobot.03 with yfinance library
#
#   12/05/22 - Forked version for Tonka_Prdtn.
#   12/05/22 - Added tickers with high Quant rating. Tightened limits for best_options.
#   11/27/22 - Remove IEX. Now using Darqube for market data. Added many more tickers, approx. 150 total.
#
# *****************
#
# USER PARAMS
# User can change these parameters as appropriate.
#
# *****************

DAR_key = 'a5e6a13d876c4819ad02083f76ff22a3' # Get one from Darqube and enter it here, between the single quotes.

TICKERS = ('URI', 'JEF', 'BSM', 'BGSF', 'CPA', 'STLD', 'AMR', 'UNVR', 'CVCO', 'HUBG', 'ACLS', 'ASRT', \
           'AFYA', 'SMCI', 'HDSN', 'VIST', 'VRNA', 'NWG', 'PINE', 'VIR', 'ASO', 'PERI', 'STRL', 'VPG', \
           'AAPL', 'ABBV', 'AFL', 'ADBE', 'AIG', 'AMD', 'AMZN', 'AOS', 'ARNC', 'AVB', 'AVGO', \
           'AZO', 'BA', 'BAC', 'BBBY', 'BIIB', 'BNTX', 'CAG', 'CAT', 'CF', 'CL', \
           'CLX', 'CMCSA', 'CRM', 'CRWD', 'CSCO', 'CSX', 'CVS', 'CVX', 'DAC', 'DDOG', \
           'DHI', 'DUK', 'DVN', 'EPAM', 'EPD', 'ET', 'EXPE', 'FAF', 'FANG', 'FDX', 'FLNG', \
           'FMS', 'FNF', 'FSLY', 'FSR', 'FYBR', 'GD', 'GIS', 'GLNG', 'GOOGL', 'GS', 'GSL', 'HD', 'HON', \
           'IBM', 'INTC', 'IPI', 'IRM', 'JNJ', 'JPM', 'K', 'KO', 'LMT', 'LOW', \
           'LRCX', 'LYB', 'MA', 'MCK', 'META', 'MGA', 'MLM', 'MMM', 'MMP', 'MOS', 'MPW', \
           'MRK', 'MRNA', 'MRVL', 'MSFT', 'MU', 'NFLX', 'NKTR', 'NOW', 'NRG', 'NVDA', \
           'OKTA', 'ON', 'PANW', 'PFE', 'PHM', 'PII', 'PLTR', 'PRU', 'PYPL', 'QCOM', 'RITM', \
           'RIVN', 'ROST', 'RTX', 'SBLK', 'SBUX', 'SCHW', 'STOR', 'SKT', 'SNOW', 'SO', 'SPG', 'SPY', \
           'STWD', 'TSLA', 'TRV', 'TXN', 'TSCO', 'TSN', 'UNP', 'UPS', 'USAC', 'V', 'VICI', 'VGT', 'VMW', \
           'VNOM', 'VOO', 'WEC', 'WMT', 'WSM', 'WFC', 'XOM', 'ZIM')

HIDE_TICKERS = ('AOS', 'FSLY', 'NOW', 'PLTR', 'TSLA', 'VNOM', 'RIVN', 'NKTR') # Keep these tickers for database, but do not show them in best_options, because you do not want to own them.  

FILE_ID_TAG       = '_RBDev'

# *****************
#
# Import Libraries
#
# *****************

import csv
import copy
import json
import numpy as np
import os
import pandas as pd
import requests   # for http requests
import scipy
from   scipy import stats
from   scipy.stats import norm
import time
import yfinance as yf

import datetime
from   datetime import datetime
from   datetime import date
from   datetime import timedelta
#from   datetime import fromtimestamp

from   dateutil.relativedelta  import relativedelta
from   yahoo_earnings_calendar import YahooEarningsCalendar

#import investpy
#import glob
#import collections
#from enum import Enum
#import re     # re is regular expression. For searching strings.

# *****************
#
# Flow Control
# Internal Constants
#
# *****************

update_root_data      = True  #skip WTI call if testing code. Just use synthetic WTI_dict instead.
update_root_data_ER   = True  #skip call to yahoo calendar if testing code.
update_option_data    = True  #skip calls to yfinance if testing code.


profile = 'POWARRMax'

DATE_FORMAT       = '%Y-%m-%d %H:%M:%S'

TICKER_DELAY      = 0.020  # seconds between tickers for data requests.
EXPIRY_DELAY      = 0.005

MAX_DAYSOUT = 366.0  # This constant controls how many expiry strings are downloaded. Use low # to exclude BTOS and reduce ext. calls. Hi # includes BTOS.
MIN_DAYSOUT = 3.0

FEE_SPREAD = 0.75     # Fee in the bid/ask range.

call_ticker_options = pd.DataFrame()
put_ticker_options  = pd.DataFrame()
ticker_options      = pd.DataFrame() # for a single ticker
all_options         = pd.DataFrame() # includes options for all tickers. Do not use any limits or exclusions for all_options.
good_options        = pd.DataFrame() # Has rankings for CSTO options and BTOs (CBTO).
stats               = pd.DataFrame() # Not used now.
options_open        = pd.DataFrame() # persistent file used by OptionCloser to recommend BTC offers.
    


In [2]:
# *****************
#
# Functions to convert broker reports into standard POWARRMax report formats for holdings and xtns.
# 
# *****************

trade_log_open = pd.DataFrame()

def Broker_Converter_TDA(root_data, broker_option_holdings, broker_option_xtns):

    # *****************
    # 1. Read broker report with option holdings.  
    # 2. For open_options parse option 'contractSymbol' into root, option type (P or C), strike price, and expiry date.
    # 3. Calculate 3 dfs for at risk dashbaord 
    #
    # *****************

    # TDA Column Names
    TDA_HOLDINGS_COLS = ('Symbol', 'Qty')
    TDA_XTN_COLS      = ('DATE', 'TRANSACTION ID', 'DESCRIPTION', 'QUANTITY', 'SYMBOL', 'PRICE', 'COMMISSION', 'AMOUNT')
    TDA_DROP_COLS     = ('REG FEE', 'SHORT-TERM RDM FEE', 'FUND REDEMPTION FEE', ' DEFERRED SALES CHARGE')
    
    OPTION_XTN_TYPES = ('Buy to open', 'Sell to open', 'Buy to close', 'Sell to close', 'Assignment', 'Expired')
    OPEN_XTN_TYPES   = ('Sell to open', 'Buy to open')
    CLOSING_XTN_TYPES = ('Sell to close', 'Buy to close', 'Assignment', 'Expired')

    # POWARRMax Column Names
    TRADE_LOG_CLOSE_METRICS_COLS = ('Close ARR', 'Assigned ARR')

    TRADE_LOG_OPEN_COLS = ('Acct', 'Open Daysout', 'Expiry Date', 'Expiry Daysout', 'Open Action', 'Open Commsn' \
                           'Open Date', 'Contracts', 'Open Fee / Share', 'Open Root Price', 'Option Type', \
                           'Root', 'Strike Price', 'Open Xtn Type')
    
    TRADE_LOG_OPEN_METRICS_COLS = ('Open IV', 'Open POW', 'ARR BE Close Fee', 'ARR BE Profit', 'Cur Price % Chg', \
                                   'Cur Root Price', 'Open % Fee', 'Open % OTM', 'Open ARR', \
                                   'Strike v Cur Root Price', 'Open Net Proceeds', 'Cash at Risk', \
                                   'Stock $ at Risk')

    TRADE_LOG_CLOSE_COLS = ('Close Fee / Share', 'Close Date', 'Close Root Price', \
                            'Close Net Proceeds', 'Close Daysout', 'Close Commsn', 'Close Xtn Type')
    HOLDINGS_COLS = ('Root', 'Shares', 'Committed Shares', 'Share Price', 'Total Value', 'Expiry Date', 'Contracts', \
                     'Option Type', 'Open Action', 'Strike Price', 'Contracts', 'Cur Fee / Share')

    TRADE_LOG_COLS = TRADE_LOG_OPEN_COLS + TRADE_LOG_OPEN_METRICS_COLS + TRADE_LOG_CLOSE_COLS + TRADE_LOG_CLOSE_METRICS_COLS

    # **********
    # From brokerage holdings and activity statements, create open_options.
    
    # Create holdings df from top rows of broker_report. 
    
    broker_option_holdings_mask = holdings_row_mask = broker_option_holdings.iloc[:,0] == 'Options'
    holdings_row = broker_option_holdings[broker_option_holdings_mask]
    
    split_index = holdings_row.index.values.astype(int)[0] + 3
    holdings = broker_option_holdings[broker_option_holdings.index >= split_index].copy()
    
    # Set column headers to first row, then reset index.
    holdings.columns = holdings.iloc[0]
    holdings = holdings[1:]
    holdings.reset_index(drop = True, inplace = True)
    
    # Drop empty rows.
    holdings = holdings[holdings['Symbol'].notna()].copy()
    holdings.reset_index(drop = True, inplace = True)
    
    # Drop last row.
    last_row = holdings.shape[0] - 1
    holdings.drop(index = last_row, inplace = True)
    
    # Create open_options df. This contains all the option trades open now, both puts and calls.
    #
    open_options = pd.DataFrame()
    open_options = holdings['Symbol'].str.split(' ',expand=True)
    open_options.rename(columns = {0:'Root', 1: 'month', 2: 'day', 3: 'year', 4: 'Strike Price', 5: 'Option Type', \
                                   6: 'Freq'}, inplace = True)
    open_options = pd.concat([holdings, open_options], axis = 1).copy()
    
    # Write expiry Date as datetime and string
    open_options['month'] = pd.to_datetime(open_options.month, format='%b').dt.month.astype(str)
    open_options['Expiry Date str'] = open_options['year'] +'-'+ open_options['month'] +'-'+ open_options['day']
    open_options['Expiry Date'] = pd.to_datetime(open_options['Expiry Date str'], infer_datetime_format = True)
    
    open_options['Contracts'] = open_options['Qty']
    open_options['Contracts'] = open_options['Contracts'].astype(int)
    
    # Convert 'Put' to 'P' and 'Call' to 'C'
    open_options['Option Type'].mask(open_options['Option Type'] == 'Call', 'C', inplace = True)
    open_options['Option Type'].mask(open_options['Option Type'] == 'Put', 'P', inplace = True)
                                    
    open_options.drop(columns = ['Underlying symbol', 'Symbol', 'Qty', 'month', 'day', 'year', \
                                 'Gain ($)', 'Maint req', 'Freq', 'Expiry Date str'], inplace = True)

    # Update 'Cur Root Price' column in open_options from price in root_data. 
    open_options['Cur Root Price'] = 0.0
    print('after col. drop open options shape = ')
    print(open_options)
    
    unique_tickers = open_options['Root'].unique()
    for tick in unique_tickers:
        tick_indexes = open_options[open_options['Root'] == tick].index
        # Fails with ticker 'MU'. So, for now skip MU to keep going w/debug.
        if tick != 'MU':  # Skip MU since it fails. Fix it next.
            open_options.loc[tick_indexes, 'Cur Root Price'] = root_data.loc[tick, 'root price']

    opt_xtns = pd.DataFrame()  #Need to create this eventually.
    
    return holdings, open_options, opt_xtns

def Broker_Converter_Van(root_data, broker_report):

    # *****************
    # NOTE - This version works with the Vanguard report, which awkwardly combines holdings and transactions
    #       into a single csv file. 
    #
    # 1. Read broker report with holdings and xtns, which Vanguard combines into a single df. 
    # 2. Parse report into three dfs: stock holdings, xtns, and open option holdings (trades). 
    #     For open_options parse option 'contractSymbol' into root, option type (P or C), strike price, and expiry date.
    #
    #
    # *****************
    
    # Vanguard Column Names
    
    VGD_COLS = ('Trade Date', 'Commission Fees', 'Share Price', 'Shares', 'Account Number', \
                'Accrued Interest', 'Account Type', 'Settlement Date', 'Investment Name', \
                'Transaction Description', 'Transaction Type', 'Net Amount', 'Principal Amount', \
                'Symbol')
    
    DROP_VGD_COLS = ['Account Number', 'Settlement Date', 'Transaction Description', 'Investment Name', 'Account Type']

    VAN_HOLDINGS_COLS = ('Account Number', 'Investment Name', 'Symbol', 'Shares', 'Share Price')
    
    OPTION_XTN_TYPES = ('Buy to open', 'Sell to open', 'Buy to close', 'Sell to close', 'Assignment', 'Expired')
    OPEN_XTN_TYPES   = ('Sell to open', 'Buy to open')
    CLOSING_XTN_TYPES = ('Sell to close', 'Buy to close', 'Assignment', 'Expired')

    # POWARRMax Column Names
    TRADE_LOG_CLOSE_METRICS_COLS = ('Close ARR', 'Assigned ARR')

    TRADE_LOG_OPEN_COLS = ('Acct', 'Open Daysout', 'Expiry Date', 'Expiry Daysout', 'Open Action', 'Open Commsn' \
                           'Open Date', 'Contracts', 'Open Fee / Share', 'Open Root Price', 'Option Type', \
                           'Root', 'Strike Price', 'Open Xtn Type')
    
    TRADE_LOG_OPEN_METRICS_COLS = ('Open IV', 'Open POW', 'ARR BE Close Fee', 'ARR BE Profit', 'Cur Price % Chg', \
                                   'Cur Root Price', 'Open % Fee', 'Open % OTM', 'Open ARR', \
                                   'Strike v Cur Root Price', 'Open Net Proceeds', 'Cash at Risk', \
                                   'Stock $ at Risk')

    TRADE_LOG_CLOSE_COLS = ('Close Fee / Share', 'Close Date', 'Close Root Price', \
                            'Close Net Proceeds', 'Close Daysout', 'Close Commsn', 'Close Xtn Type')
    HOLDINGS_COLS = ('Root', 'Shares', 'Committed Shares', 'Share Price', 'Total Value', 'Expiry Date', 'Contracts', \
                     'Option Type', 'Open Action', 'Strike Price', 'Contracts', 'Cur Fee / Share')

    TRADE_LOG_COLS = TRADE_LOG_OPEN_COLS + TRADE_LOG_OPEN_METRICS_COLS + TRADE_LOG_CLOSE_COLS + TRADE_LOG_CLOSE_METRICS_COLS

    # **********
    # From brokerage holdings and activity statement, create 3 separate dfs: -- holdings, opt_xtns, and open_options.
    
    # 1. Create holdings df from top rows of broker_report.
    xtns_row = broker_report.loc[broker_report['colx1'] == 'Trade Date']
    split_index = xtns_row.index.values.astype(int)[0] 
    holdings = broker_report[broker_report.index < split_index].copy()
   
    # Remove unused columns from holdings
    unused_cols = []
    for i in range(5, 15):
        col_name = 'colx' + str(i)
        unused_cols.append(col_name)
    
    holdings.drop(columns = unused_cols, inplace = True)
    
    # restore oroginal column headers, from row 1 and delete 'colx' headers
    holdings.rename(columns = holdings.iloc[0], inplace = True)
    holdings.drop(index = 0, inplace = True)
    holdings.drop(columns = 'Account Number', inplace = True)
    
    # 2. Create opt_xtns df from brokerage report. 
    #
    xtns = broker_report[broker_report.index >= split_index].copy()

    # Rename columns in xtns and reset index.
    xtns.columns = xtns.iloc[0]
    xtns = xtns[1:]
    xtns.reset_index(drop = True, inplace = True)

    # Add trade_log column names to xtns
    xtns = xtns.reindex(columns = [*xtns.columns.tolist(), *TRADE_LOG_COLS], fill_value = None)
    
    # Select option trades only.
    opt_xtns = xtns[xtns['Transaction Type'].isin(OPTION_XTN_TYPES)].copy()
        
    opt_xtns.drop(DROP_VGD_COLS, axis = 1, inplace=True)
    opt_xtns.reset_index(drop = True, inplace = True)
        
    # 3. Create open_options df. This contains all the option trades open now, both puts and calls.
    #
    open_options = pd.DataFrame()
    open_options = holdings['Symbol'].str.split(' ',expand=True)
    open_options.rename(columns = {0:'Root', 1: 'Expiry Date', 2: 'Option Type', 3: 'Strike Price'}, inplace = True)
    open_options = pd.concat([holdings, open_options], axis = 1).copy()

    open_options = open_options[open_options['Option Type'].isin(['P', 'C'])].copy()
    open_options['Contracts'] = open_options['Shares']
    open_options['Contracts'] = open_options['Contracts'].astype(int)
    
    open_options.drop(columns = ['Investment Name', 'Symbol', 'Shares'], inplace = True)
    
#    Update 'Cur Root Price' column in open_options from price in root_data. 
#    print(root_data.head(10))
    open_options['Cur Root Price'] = 0.0
    unique_tickers = open_options['Root'].unique()
    for tick in unique_tickers:
        tick_indexes = open_options[open_options['Root'] == tick].index
    return holdings, open_options, opt_xtns

#************
# Main Program calls Broker_Converter
#************

# Main program tests Broker_Converter ONLY. It uses three test files: trade_log_open, trade_log_closed,
#      and opt_xtns. It reads these directly. 
root_data        = pd.read_csv('root_data.csv')
root_data.sort_values('Ticker', axis = 0, inplace = True, ignore_index = True)
root_data.set_index('Ticker', inplace = True)

BROKERAGE = 'Skip' 

if BROKERAGE == 'TDA':
    broker_open_options = pd.read_csv('TDA_positions.csv')
    broker_xtns         = pd.read_csv('TDA_xtns.csv')
    holdings, open_options, opt_xtns = Broker_Converter_TDA(root_data, broker_open_options, broker_xtns)

if BROKERAGE == 'Vanguard':
    # Create temporary column headersbecause Vanguard concatenates the positions report and the transactions report.
    col_headers = []
    for i in range(15):
        col_name = 'colx' +str(i)
        col_headers.append(col_name)
        
    broker_report_van = pd.read_csv('broker_report_van.csv',header = None, names = col_headers, skip_blank_lines = True)
    
    holdings, open_options, opt_xtns = Broker_Converter_Van(root_data, broker_report_van)

if BROKERAGE == 'Skip':  # Use for testing. Skip parsing brokerage reports and use files in the repo.
    holdings      = pd.read_csv('broker_holdings.csv')
    open_options = pd.read_csv('broker_open_options.csv')
    opt_xtns     = pd.read_csv('broker_opt_xtns.csv')
    
    print(' Skipped broker converters. Read files already in the repo.')

else:   # Write updated files to the repo.
    holdings.to_csv('broker_holdings.csv', index = False)
    open_options.to_csv('broker_open_options.csv', index = False)
    opt_xtns.to_csv('broker_opt_xtns.csv', index = False)

    print('wrote 3 results files for Broker_Converter.')


 Skipped broker converters. Read files already in the repo.


In [ ]:
# *****************
# Get option data.
#
# 1. Get real-time price data from Darqube. Write to root_data. 
# 2. Get ER dates from Yahoo Earnings Calendar and write to root_data.
# 3. Get option strings from yfinance. Use "try" statements to skip over exp dates where call to yfinance failed.
# 4. Parse the option data fields into the common POWMax formats.
# 
# *****************

#Use Darqube for stock price data - both market and historical.

root_data = pd.read_csv('root_data.csv')
root_data.set_index('Ticker', inplace = True, drop = True)

print('start Darqube access at:  ', datetime.now(), '  Done in ',len(TICKERS) * 1.0 / 60.0, ' minutes.')
for ticker in TICKERS:

    price_rqst = 'https://api.darqube.com/data-api/market_data/quote/' + ticker + '?token=' + DAR_key
    response = requests.get(price_rqst)
    price_dict = response.json()
    root_data.loc[ticker, 'root price'] = price_dict['price']

root_data.sort_index(inplace = True) #Save root_data with updated ER dates.
root_data.to_csv('root_data.csv')
print(datetime.now())

if update_root_data_ER:                # skip call to YEC for ER update
    days_to_check_ER = (1, 4)  #check for ER updates when day # matches any in this list. This combo guarantees ER check on at least one date each month. 
    print('today date = ', date.today().day)
    if date.today().day in days_to_check_ER:
        yec_custom_delay_s = 0.1
        yec = YahooEarningsCalendar(yec_custom_delay_s)
        print('      Just updating ER dates.')

        for ticker in TICKERS:
            try:
                next_er_date = (datetime.fromtimestamp(yec.get_next_earnings_date(ticker)).strftime('%Y-%m-%d %H:%M'))
            except: 
                print(ticker, ' got retrieval error')
            else:
                root_data.loc[ticker, 'ER Date'] = next_er_date
                print(ticker, root_data.loc[ticker, 'ER Date'])
        root_data.sort_index(inplace = True) #Save root_data with updated ER dates.
        root_data.to_csv('root_data.csv')

datetoday = date.today()
#root_data['ER daysout'] = np.where(root_data['ER Date'] != None,(root_data['ER Date'] - pdatetoday).datetime.days, NaN) # convert these to date code to subtract

root_data.sort_index(inplace = True)
root_data.to_csv('root_data.csv')

if update_option_data == False: # then skip the actual data collection from the net. Just grab the all_options file and calculate good_options.
#    read_file = 'all_options' + FILE_ID_TAG + '.csv'
    read_file = 'all_options' + '2022-03-22' + '.csv'
    all_options = pd.read_csv(read_file)
    
    print('skipped yahoo options update.')
else:
    for ticker in TICKERS:
        print('getting data for ', ticker, ' at time ', datetime.now())
        try:                                      # get option strings from yfinance for each ticker.
                                                  # if call to yfinance fails, then just skip this expiry date.
            ticker_data = yf.Ticker(ticker)       # get option strings from yfinance for each ticker.
        except:
            print('yfinance call failed for ', ticker) 
        quote_time = datetime.now() + timedelta(hours = 3)
        quote_date = date.today()
        expiry_dates = ticker_data.options
        call_ticker_options = pd.DataFrame()
        put_ticker_options  = pd.DataFrame()
        for exp_date in expiry_dates:
            exp_daysout = (datetime.strptime(exp_date, '%Y-%m-%d').date() - quote_date).days
            if (exp_daysout < MAX_DAYSOUT and exp_daysout > MIN_DAYSOUT):
                #if exp_date != '2020-04-02': #test code only. For normal operation delete this.
                    
                    #If doing BTOs set MAX_DAYSOUT to high number, say 1000.0.
                    try:
                        option_data = ticker_data.option_chain(exp_date)
                        call_ticker_options = pd.concat([call_ticker_options, option_data.calls], sort = False, ignore_index = True)
                        put_ticker_options  = pd.concat([put_ticker_options,  option_data.puts],  sort = False, ignore_index = True)
                    except:
                        print('     yfinance call failed for expiry ', exp_date)
                    time.sleep(EXPIRY_DELAY)           
        call_ticker_options['option_type'] = 'call'
        put_ticker_options['option_type'] = 'put'

        # calculate columns unique to each ticker        
        ticker_options = pd.concat([call_ticker_options, put_ticker_options], sort = False, ignore_index = True)
        ticker_options['root'] =  ticker
        ticker_options['root price'] = root_data.loc[ticker, 'root price']
        ticker_options['Quote_Time'] = quote_time
        ticker_options['ER Date'] = root_data.loc[ticker, 'ER Date']
        ticker_options['Expstring'] = '20' + ticker_options['contractSymbol'].str[len(ticker):len(ticker) + 6]
        ticker_options['Expiry'] = ticker_options['Expstring'].str[:4] + '-' + ticker_options['Expstring'].str[4:6] + '-' + \
                                   ticker_options['Expstring'].str[6:8]
        all_options = pd.concat([all_options, ticker_options], sort = False, ignore_index = True)

        time.sleep(TICKER_DELAY)

    print('Got all the data for all tickers. Created additional cols. time = ', datetime.now())


In [ ]:
#****************
# Select best options based on selected profile. Currently the only profile allowed is 
#*****************

best_STO = all_options.copy()


if profile == 'POWARRMax':
    # CONSTANTS for best_option ranges
    POWmin     = 69
    POWmax     = 85
    ARRmin     = 19.
    ARRmax     = 95.
    PctOTMmin  =  4.0
    PctOTMmax  = 18.0
    PctFeemin  =  2.0
    PctFeemax  = 12.0
    daysoutmax = 55.0
    daysoutmin =  2.0

if profile == 'LEAP':
    # CONSTANTS for best_option ranges for long dated puts.
    POWmin     = 5
    POWmax     = 85
    ARRmin     = 10
    ARRmax     = 1000
    PctOTMmin  =  -100
    PctOTMmax  = 250
    PctFeemin  =  1.8
    PctFeemax  = 100.0
    daysoutmax = 750.0
    daysoutmin =  75.0

best_STO = best_STO[(((best_STO['POW']    >= POWmin)    & (best_STO['POW']    <= POWmax))    & 
                     ((best_STO['ARR']    >= ARRmin)    & (best_STO['ARR']    <= ARRmax)))   &
                    (((best_STO['PctFee'] >= PctFeemin) & (best_STO['PctFee'] <= PctFeemax)) & 
                     ((best_STO['PctOTM'] >= PctOTMmin) & (best_STO['PctOTM']  <= PctOTMmax)))].copy()

best_STO = best_STO[(best_STO['daysout']  >= daysoutmin) & (best_STO['daysout'] <= daysoutmax)].copy() 

best_STO = best_STO[~best_STO['root'].isin(HIDE_TICKERS)].copy()

#convert some columns to int

int_columns = ['daysout', 'ARR', 'POW', 'impliedVolatility']
best_STO[int_columns] = best_STO[int_columns].astype(int)
                           
# remove temporary columns in best_STO
drop_columns = ['contractSymbol', 'inTheMoney', 'lastTradeDate', 'lastPrice', 'bid', 'ask', \
                'change', 'percentChange', 'callPITM', 'BidAskSpread', \
                'volume', 'openInterest', 'Quote_Time', 'Expstring', 'contractSize', 'currency']

#Removed ER daysout and ER-Expiry from drop_columns. Putthe back after fixing the format problem.

best_STO.drop(labels = drop_columns, axis = 1, inplace = True)

#Sort your favorite way

best_STO.sort_values(['root', 'option_type', 'POW'], ascending = [True, True, False], inplace = True)

write_file = 'best_STO' + FILE_ID_TAG + '.csv'
best_STO.to_csv(write_file, index = False, float_format = '%.2f')


print('Wrote best_STO. Finished at ', datetime.now())


In [3]:
#**********
#
# Temporary File Setup for little option guns.
# 1. Read best_options_suffix into best_options df for input to little Option guns.
# 2. Create trade_log_open. Calc all addl columns. 
#      This code block goes back into TradeManager in the trade_log_open block


In [4]:
#**********
#
# Put_STO_Gun_01
# Started 12/14/22
# 1. Develop concentrations list from open_options.
# 2. Define max_concentration rule.
# 3. Define max_cash_total and max_cash_daily rules.
# 4. Define good_company rule.
# 5. Define priority for ranking best_options.
#
#**********

def Put_STO_Gun(holdings, best_options):
    PSTO_options = best_options[best_options['option type'] == 'Put']
    return PSTO_options

Run_PSTO = True

if Run_PSTO:
    PSTO_options = Put_STO_Gun(holdings, best_options)
    write_file = 'PSTO_options_' + FILE_ID_TAG + '.csv'
    PSTO_options.to_csv(write_file)

NameError: name 'best_options' is not defined

In [ ]:
#**********
#
# Call_STO_Gun_01
# Startet 12/14/22
# 1. From holdings, create list of holdings with > 100 shares.
# 2. Calculate # contracts possible
# 3. Subtract all contracts already used in open_options
# 4. Screen best_options for only the roots with contracts available
#
#**********

def Call_STO_Gun(holdings, best_options):
    return CSTO_options

Run_CSTO = True
if Run_CSTO:
    CSTO_options = Call_STO_Gun(holdings, best_options)
    write_file = 'CSTO_options_' + FILE_ID_TAG + '.csv'
    CSTO_options.to_csv(write_file)

In [ ]:
#**********
#
# Put_BTC_Gun_01
# Started 12/14/22
# 1. From open_options create list of open puts
# 2. Screen all_options for only each open put.
# 2.1 Find matching PBTC trade. From all_options, get last price  
# 3. Annotate the put record with ARRBE price, < 20% price, etc. 
# 4. Make recommendation - Wait, place BTC for day / GTC, at price X, Wait for price drop to close, 
#        Wait for Expiry, Wait for Assignment.
#
# Make close offer IF:
#   - matching_PBTC_fee < MIN_BID_Pct or > ARRBE (with factor)  # for profitable trades
#           or concentration too high, or ITM is low enough, or cost_basis is higher than strike, or don't want the root.               
#**********

def Put_BTC_Gun(holdings, open_options, all_options):
    # Select only Put open trades
    PBTC_trades = open_options[open_options['option_type'] == 'Put'].copy()
    PBTC_trades.reset_index(drop = True, inplace = True)
    PBTC_trades['row_num'] = PBTC_trades.index
    
    #Find matching PBTC trade in all_options
    for i in range(PBTC_trades.shape[0]):
        strike      = PBTC_trades.loc['row_num' == i, 'strike']
        expiry      = PBTC_trades.loc['row_num' == i, 'expiry']
        root        = PBTC_trades.loc['row_num' == i, 'root']
        option_type = PBTC_trades.loc['row_num' == i, 'option_type']

        matching_PBTC = all_options.np.where[((all_options['strike'] == strike) & (all_options[expiry] == expiry)) \
                                           & ((all_options['option_type'] == option_type) & (all_options['root'] == root))]
        
        # Get last price, IV, close from matching PBTC
        PBTC_trades.loc['row_num' == i, 'close_IV'] = matching_PBTC.loc[0, 'Implied Volatility']
        PBTC_trades.loc['row_num' == i, 'close_last price'] = matching_PBTC.loc[0, 'last price']
        PBTC_trades.loc['row_num' == i, 'bid_fee'] = matching_PBTC.loc[0, 'fee']
    
    #Calculate vector column values for PBTC_trades columns
    PBTC_trades['ARRBE'] = days / price
    PBTC_trades['min_close_bid'] = MIN_BID_Pct * PBTC_trades['fee']
    PBTC_trades['est_close_ARR'] = MIN_ARR_Pct * est_close_ARR

    # Add explanation.
    
    return PBTC_options

Run_PBTC = True

if Run_PBTC:
    PBTC_options = Put_BTC_Gun(holdings, best_options)
    write_file = 'PBTC_options_' + FILE_ID_TAG + '.csv'
    PBTC_options.to_csv(write_file)

In [ ]:
#**********
#
# Call_BTC_Gun_01
# Started 12/14/22
# 1. From open_options create list of open_calls
# 2. Screen all_options for each of the open_calls
# 3. Annotate the call with ARRBE price, < 20% price, etc. 
#
#**********

def Call_BTC_Gun(holdings, all_options):
    
    return CBTC_options

Run_CBTC = True

if Run_CBTC:
    CBTC_options = Call_BTC_Gun(holdings, best_options)
    write_file = 'CBTC__options_' + FILE_ID_TAG + '.csv'
    CBTC_options.to_csv(write_file)
    
print('shot the little guns.')
print()
    
stop

In [ ]:
# ****************
#
#Calculate metrics for each option.
#
# *****************
all_options['Quote_Time'] = pd.to_datetime(all_options['Quote_Time'], format = DATE_FORMAT)
all_options['Expiry']     = pd.to_datetime(all_options['Expiry'], format = DATE_FORMAT)
all_options['fee']        = FEE_SPREAD * (all_options['ask'] - all_options['bid']) + all_options['bid']
all_options['daysout']    = (all_options['Expiry'] - all_options['Quote_Time']) / np.timedelta64(1,'D')
all_options['strike']     = pd.to_numeric(all_options['strike'], errors = 'coerce')
all_options['root price'] = pd.to_numeric(all_options['root price'], errors = 'coerce')
#all_options['ER Date']    = pd.to_datetime(all_options['ER Date'], format = DATE_FORMAT)
#all_options['ER daysout'] = (all_options['ER Date'] - all_options['Quote_Time']) / np.timedelta64(1,'D')
#all_options['ER-Expiry']  = all_options['ER daysout'] - all_options['daysout']

all_options['OTM']     = np.where(all_options['option_type'] == 'call', all_options['strike'] - all_options['root price'], \
                                  all_options['root price'] - all_options['strike'])

# use np.where to calculate different ARRs based on put/call and buy/sell
all_options['ARR']     = 100.0 * (all_options['fee'] * 365) / (all_options['daysout'] * all_options['root price'])
all_options['PctOTM']  = 100.0 * all_options['OTM'] / all_options['root price']

#all_options['TQI']     = all_options['ARR'] * all_options['PctOTM'] /10

all_options['callPITM'] = 100.0 * norm.cdf(np.log(all_options['root price'] / all_options['strike']) /
                                  (all_options['impliedVolatility'] * (all_options['daysout'] / 365)**(1/2)))

all_options['POW'] = np.where(all_options['option_type'] == 'call', 100.0 - all_options['callPITM'], all_options['callPITM'])

all_options['PctFee'] = 100.0 * all_options['fee'] / all_options['root price']
all_options['BidAskSpread'] = 100.0 * (all_options['ask'] - all_options['bid']) / all_options['bid']
all_options['impliedVolatility'] = 100.0 * all_options['impliedVolatility']

write_file = 'all_options' + FILE_ID_TAG + '.csv'
all_options.to_csv(write_file, index = False)
print('Wrote all_options')

